# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Schirmer  ->  M. Schirmer  |  ['M. Schirmer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
N. Winkel  ->  N. Winkel  |  ['N. Winkel']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Liu  ->  J. Liu  |  ['J. Liu']
R. Zhang  ->  R. Zhang  |  ['R. Zhang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 75 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.02487


extracting tarball to tmp_2411.02487...

 done.


Found 56 bibliographic references in tmp_2411.02487/Euclid_Gaia_Paper_I_arXiv.bbl.
Retrieving document from  https://arxiv.org/e-print/2411.02488


...

 done.
Retrieving document from  https://arxiv.org/e-print/2411.03126


extracting tarball to tmp_2411.03126...

 done.
Retrieving document from  https://arxiv.org/e-print/2411.03184


extracting tarball to tmp_2411.03184... done.


Found 67 bibliographic references in tmp_2411.03184/darkaccretion.bbl.
Issues with the citations
syntax error in line 137: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.02487-b31b1b.svg)](https://arxiv.org/abs/2411.02487) | **Euclid: High-precision imaging astrometry and photometry from Early Release Observations. I. Internal kinematics of NGC 6397 by combining Euclid and Gaia data**  |
|| M. Libralato, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-11-06*|
|*Comments*| *23 pages, 21 figures. Accepted for publication in A&A on October 24, 2024. Astro-photometric catalogs and stacked images will be available at the CDS after the paper will be published*|
|**Abstract**|            The instruments at the focus of the Euclid space observatory offer superb, diffraction-limited imaging over an unprecedented (from space) wide field of view of 0.57 deg$^2$. This exquisite image quality has the potential to produce high-precision astrometry for point sources once the undersampling of Euclid's cameras is taken into account by means of accurate, effective point spread function (ePSF) modelling. We present a complex, detailed workflow to simultaneously solve for the geometric distortion (GD) and model the undersampled ePSFs of the Euclid detectors. Our procedure was successfully developed and tested with data from the Early Release Observations (ERO) programme focused on the nearby globular cluster NGC 6397. Our final one-dimensional astrometric precision for a well-measured star just below saturation is 0.7 mas (0.007 pixel) for the Visible Instrument (VIS) and 3 mas (0.01 pixel) for the Near-Infrared Spectrometer and Photometer (NISP). Finally, we present a specific scientific application of this high-precision astrometry: the combination of Euclid and Gaia data to compute proper motions and study the internal kinematics of NGC 6397. Future work, when more data become available, will allow for a better characterisation of the ePSFs and GD corrections that are derived here, along with assessment of their temporal stability, and their dependencies on the spectral energy distribution of the sources as seen through the wide-band filters of Euclid.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.02488-b31b1b.svg)](https://arxiv.org/abs/2411.02488) | **Combining Direct Black Hole Mass Measurements and Spatially Resolved Stellar Kinematics to Calibrate the $M_{\rm BH}$-$\sigma_\star$ Relation of Active Galaxies**  |
|| <mark>N. Winkel</mark>, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-11-06*|
|*Comments*| *Resubmitted to ApJ after initial positive referee report. 40 pages, 19 figures, 6 tables*|
|**Abstract**|            The origin of the tight scaling relation between the mass of supermassive black holes (SMBHs; $M_{\rm BH}$) and their host-galaxy properties remains unclear. Active galactic nuclei (AGNs) probe phases of ongoing SMBH growth and offer the only opportunity to measure $M_{\rm BH}$ beyond the local Universe. However, determining AGN host galaxy stellar velocity dispersion $\sigma_\star$, and their galaxy dynamical masses $M_{\rm dyn}$, is complicated by AGN contamination, aperture effects and different host galaxy morphologies. We select a sample of AGNs for which $M_{\rm BH}$ has been independently determined to high accuracy by state-of-the-art techniques: dynamical modeling of the reverberation signal and spatially resolving the broad-line region with VLTI/GRAVITY. Using IFU observations, we spatially map the host galaxy stellar kinematics across the galaxy and bulge effective radii. We find that that the dynamically hot component of galaxy disks correlates with $M_{\rm BH}$; however, the correlations are tightest for aperture-integrated $\sigma_\star$ measured across the bulge. Accounting for the different $M_{\rm BH}$ distributions, we demonstrate - for the first time - that AGNs follow the same $M_{\rm BH}$-$\sigma_\star$ and $M_{\rm BH}$-$M_{\rm bulge, dyn}$ relations as quiescent galaxies. We confirm that the classical approach of determining the virial factor as sample-average, yielding ${\rm log }f= 0.65 \pm 0.18$, is consistent with the average $f$ from individual measurements. The similarity between the underlying scaling relations of AGNs and quiescent galaxies implies that the current AGN phase is too short to have altered BH masses on a population level. These results strengthen the local calibration of $f$ for measuring single-epoch $M_{\rm BH}$ in the distant Universe.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.03184-b31b1b.svg)](https://arxiv.org/abs/2411.03184) | **The Soltan argument at $z=6$: UV-luminous quasars contribute less than 10% to early black hole mass growth**  |
|| <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-11-06*|
|*Comments*| *Submitted to Open Journal of Astrophysics; 10 pages, 5 figures*|
|**Abstract**|            We combine stellar mass functions and the recent first JWST-based galaxy-black hole scaling relations at $z=6$ to for the first time compute the supermassive black hole (SMBH) mass volume density at this epoch, and compare this to the integrated SMBH mass growth from the population of UV-luminous quasars at $z>6$. We show that even under very conservative assumptions almost all growth of SMBH mass at $z>6$ does not take place in these UV-luminous quasars, but must occur in systems obscured through dust and/or with lower radiative efficiency than standard thin accretion disks. The `Soltan argument' is not fulfilled by the known population of bright quasars at $z>6$: the integrated SMBH mass growth inferred from these largely unobscured active galactic nuclei (AGN) in the early Universe is by a factor $\ge$10 smaller than the total SMBH mass volume density at $z=6$. This is valid under a large range of assumption about luminosity, mass functions, and accretion modes, and is likely still a factor >2 smaller when accounting for known obscuration fractions at this epoch. The resulting consequences are: >90%, possibly substantially more, of SMBH-buildup in the early Universe does not take place in luminous unobscured quasar phases, but has to occur in obscured systems, with dust absorbing most of the emitted UV-visible AGN emission, potentially with accretion modes with super-Eddington specific accretion rates. This is consistent with short lifetimes for luminous quasar phases from quasar proximity zone studies and clustering. This would remove the empirical need for slow SMBH growth and hence exotic `high-mass seed' black holes at early cosmic time. It also predicts a large population of luminous but very obscured lower-mass quasars at $z>6$, possibly the JWST `Little Red Dots'. This finding might severe impact on how we will diagnose SMBH growth at $z=7$ to 15 in the future.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.02497-b31b1b.svg)](https://arxiv.org/abs/2411.02497) | **Asymmetries and Circumstellar Interaction in the Type II SN 2024bch**  |
|| J. E. Andrews, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-11-06*|
|*Comments*| *Submitted to ApJ*|
|**Abstract**|            We present a comprehensive multi-epoch photometric and spectroscopic study of SN 2024bch, a nearby (19.9 Mpc) Type II supernova (SN) with prominent early high ionization emission lines. Optical spectra from 2.9 days after the estimated explosion reveal narrow lines of H I, He II, C IV, and N IV that disappear by day 6. High cadence photometry from the ground and TESS show that the SN brightened quickly and reached a peak M$_V \sim$ $-$17.8 mag within a week of explosion, and late-time photometry suggests a $^{56}$Ni mass of 0.050 M$_{\odot}$. High-resolution spectra from day 8 and 43 trace the unshocked circumstellar medium (CSM) and indicate a wind velocity of 30--40 km s$^{-1}$, a value consistent with a red supergiant (RSG) progenitor. Comparisons between models and the early spectra suggest a pre-SN mass-loss rate of $\dot{M} \sim 10^{-3}-10^{-2}\ M_\odot\ \mathrm{yr}^{-1}$, which is too high to be explained by quiescent mass loss from RSGs, but is consistent with some recent measurements of similar SNe. Persistent blueshifted H I and [O I] emission lines seen in the optical and NIR spectra could be produced by asymmetries in the SN ejecta, while the multi-component H$\alpha$ may indicate continued interaction with an asymmetric CSM well into the nebular phase. SN 2024bch provides another clue to the complex environments and mass-loss histories around massive stars.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.02743-b31b1b.svg)](https://arxiv.org/abs/2411.02743) | **When LAMOST meets Gaia DR3 Exploring the metallicity of open clusters**  |
|| <mark>R. Zhang</mark>, et al. |
|*Appeared on*| *2024-11-06*|
|*Comments*| **|
|**Abstract**|            Context. Open clusters (OCs) are valuable probes of stellar population characteristics. Their age and metallicity provide insights into the chemical enrichment history of the Milky Way. By studying the metallicity of OCs, we can explore the spatial distribution of composition across the Galaxy and understand stellar birth radii through chemical tagging. However, inferring the original positions of OCs remains a challenge. Aims. This study investigates the distribution of metallicity in the solar neighborhood using data from Gaia DR3 and LAMOST spectra. By measuring accurate ages and metallicities, we aim to derive birth radii and understand stellar migration patterns. Methods. We selected 1131 OCs within 3 kpc of the Sun from Gaia DR3 and LAMOST DR8 low-resolution spectra (R=1800). To correct the LAMOST data, we incorporated high-resolution spectra from GALAH DR3 (R=28000) using an artificial neural network. The average metallicity of the OCs was derived from reliable [Fe/H] values of their members. We examined the metallicity distribution across the Galaxy and calculated birth radii based on age and metallicity. Results. The correction method reduces the systematic offset in LAMOST data. We found a metallicity gradient as a function of Galactocentric distance and guiding radii. Comparisons with chemo-dynamic simulations show that observed metallicity values are slightly lower than predicted when uncertainties are ignored, but the metallicity gradients align with previous studies. We also inferred that many OCs near the Sun likely originated from the outer Galactic disk.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.03126-b31b1b.svg)](https://arxiv.org/abs/2411.03126) | **Combining strongly lensed and unlensed fast radio bursts: to be a more precise late-universe probe**  |
|| J.-G. Zhang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-11-06*|
|*Comments*| *18 pages, 7 figures*|
|**Abstract**|            The Macquart relation and time-delay cosmography are now two promising ways to fast radio burst (FRB) cosmology. In this work, we propose a joint method that combines strongly lensed and unlensed FRBs for improving cosmological parameter estimation by using simulated FRB data from the future sensitive coherent all-sky monitor survey, which is expected to detect a large number of FRBs including galaxy-galaxy strongly lensed events. We find that using a detectable sample of 100,000 localized FRBs including $40$ lensed events can simultaneously constrain the Hubble constant and the equation of state of dark energy, with high precision of $\varepsilon(H_0)=0.4\%$ and $\varepsilon(w)=4.5\%$ in the simplest dynamical dark energy model. The joint analysis of unlensed and lensed FRBs significantly improves the constraint on $H_0$, which could be more effective than combining either the unlensed FRBs with future gravitational wave (GW) standard sirens or the lensed FRBs with CMB. Furthermore, combining the full FRB sample with the CMB+BAO+SNe data yields $\sigma(H_0)=0.29~{\rm km~s^{-1}~Mpc^{-1}}$, $\sigma(w_0)=0.046$, and $\sigma(w_a)=0.15$ in the two-parameter dynamical dark energy model, which outperform the results from the CMB+BAO+SNe+GW data. This reinforces the cosmological implications of a multi-wavelength observational strategy in optical and radio bands. We conclude that the future FRB observations will shed light on the nature of dark energy and also the Hubble tension if enough events with long-duration lensing are incorporated.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.02487/./Figures/det001_vis_before.png', 'tmp_2411.02487/./Figures/det001_vis_after.png', 'tmp_2411.02487/./Figures/1drms.png', 'tmp_2411.02487/./Figures/det001_Y_before.png', 'tmp_2411.02487/./Figures/det001_Y_after.png']
copying  tmp_2411.02487/./Figures/det001_vis_before.png to _build/html/
copying  tmp_2411.02487/./Figures/det001_vis_after.png to _build/html/
copying  tmp_2411.02487/./Figures/1drms.png to _build/html/
copying  tmp_2411.02487/./Figures/det001_Y_before.png to _build/html/
copying  tmp_2411.02487/./Figures/det001_Y_after.png to _build/html/
exported in  _build/html/2411.02487.md
    + _build/html/tmp_2411.02487/./Figures/det001_vis_before.png
    + _build/html/tmp_2411.02487/./Figures/det001_vis_after.png
    + _build/html/tmp_2411.02487/./Figures/1drms.png
    + _build/html/tmp_2411.02487/./Figures/det001_Y_before.png
    + _build/html/tmp_2411.02487/./Figures/det001_Y_after.png
found figures ['tmp_2411.02488/./Figures/03_Results/Mbh-sigma_cor

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\hstfull}{\HST\xspace}$
$\newcommand{\hst}{HST\xspace}$
$\newcommand{\jwstfull}{\textit{James Webb} Space Telescope\xspace}$
$\newcommand{\jwst}{JWST\xspace}$
$\newcommand{\gaia}{\textit{Gaia}\xspace}$
$\newcommand{\euclid}{\textit{Euclid}\xspace}$
$\newcommand{\qfit}{\texttt{QFIT}\xspace}$
$\newcommand{\radxs}{\texttt{RADXS}\xspace}$
$\newcommand{\hpass}{\texttt{hst1pass}\xspace}$
$\newcommand{\jpass}{\texttt{jwst1pass}\xspace}$
$\newcommand{\epass}{\texttt{euclid1pass}\xspace}$
$\newcommand{\kstwo}{\texttt{KS2}\xspace}$
$\newcommand{\masyr}{mas yr^{-1}\xspace}$
$\newcommand{\egpm}{\mbox{\textit{E}-\textit{G}}\xspace}$
$\newcommand{\addtext}[1]{\textcolor{OliveGreen}{\uwave{#1}}}$</div>



<div id="title">

# $\euclid$: High-precision imaging astrometry and photometry from Early Release Observations$\thanks{This paper is published on behalf of the Euclid Consortium.}$: I. Internal kinematics of NGC 6397 by combining $\euclid$ and $\gaia$ data

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.02487-b31b1b.svg)](https://arxiv.org/abs/2411.02487)<mark>Appeared on: 2024-11-06</mark> -  _23 pages, 21 figures. Accepted for publication in A&A on October 24, 2024. Astro-photometric catalogs and stacked images will be available at the CDS after the paper will be published_

</div>
<div id="authors">

M. Libralato, et al. -- incl., <mark>M. Schirmer</mark>, <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The instruments at the focus of the $\euclid$ space observatory offer superb, diffraction-limited imaging over an unprecedented (from space) wide field of view of 0.57 deg $^2$ . This exquisite image quality has the potential to produce high-precision astrometry for point sources once the undersampling of $\euclid$ 's cameras is taken into account by means of accurate, effective point spread function (ePSF) modelling. We present a complex, detailed workflow to simultaneously solve for the geometric distortion (GD) and model the undersampled ePSFs of the $\euclid$ detectors. Our procedure was successfully developed and tested with data from the Early Release Observations (ERO) programme focused on the nearby globular cluster NGC 6397.  Our final one-dimensional astrometric precision for a well-measured star just below saturation is 0.7 mas (0.007 pixel) for the Visible Instrument (VIS) and 3 mas (0.01 pixel) for the Near-Infrared Spectrometer and Photometer (NISP). Finally, we present a specific scientific application of this high-precision astrometry: the combination of $\euclid$ and $\gaia$ data to compute proper motions and study the internal kinematics of NGC 6397. Future work, when more data become available, will allow for a better characterisation of the ePSFs and GD corrections that are derived here, along with assessment of their temporal stability, and their dependencies on the spectral energy distribution of the sources as seen through the wide-band filters of $\euclid$ .

</div>

<div id="div_fig1">

<img src="tmp_2411.02487/./Figures/det001_vis_before.png" alt="Fig9.1" width="50%"/><img src="tmp_2411.02487/./Figures/det001_vis_after.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** GD maps for the VIS quadrant 1-1.E before (left panel) and after (right panel) applying the polynomial correction. Vectors are magnified by a factor of 50 (left) and 10 000 (right) to enhance the details. The positional $x$ and $y$ positional residuals as a function of $x$ and $y$ raw VIS positions are shown in the side panels. (*fig:VISGDdet1*)

</div>
<div id="div_fig2">

<img src="tmp_2411.02487/./Figures/1drms.png" alt="Fig1" width="100%"/>

**Figure 1. -** Oned dimensional (1D) positional RMS (expressed in units of VIS or NISP pixel depending on the panel) as a function of VIS or NISP instrumental magnitude. The red horizontal line is set at the median value of bright, well-measured ($\qfit$$<$ 0.05) stars that lie within the two red, dashed, vertical lines. The median values in pixels and mas are reported in each panel. Only 20\% of the points are shown for clarity. (*fig:1drms*)

</div>
<div id="div_fig3">

<img src="tmp_2411.02487/./Figures/det001_Y_before.png" alt="Fig10.1" width="50%"/><img src="tmp_2411.02487/./Figures/det001_Y_after.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Similar to Fig. \ref{fig:VISGDdet1}, but for the NISP DET11 detector with $\YE$-filter data. Units are now NISP pixels. (*fig:NISPYGDdet1*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.02487"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand\aa{@element@tr}$</div>



<div id="title">

# Combining Direct Black Hole Mass Measurements and Spatially Resolved Stellar Kinematics to Calibrate the $\mbh$-$\sigstar$ Relation of Active Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.02488-b31b1b.svg)](https://arxiv.org/abs/2411.02488)<mark>Appeared on: 2024-11-06</mark> -  _Resubmitted to ApJ after initial positive referee report. 40 pages, 19 figures, 6 tables_

</div>
<div id="authors">

Nico~Winkel, et al.

</div>
<div id="abstract">

**Abstract:** The origin of the tight scaling relation between the mass of supermassive black holes (SMBHs; $\mbh$ ) and their host-galaxy properties remains unclear.    Active galactic nuclei (AGNs) probe phases of ongoing SMBH growth and offer the only opportunity to measure $\mbh$ beyond the local Universe. However, determining AGN host galaxy stellar velocity dispersion $\sigstar$ , and their galaxy dynamical masses $\mdyn$ , is complicated by AGN contamination, aperture effects and different host galaxy morphologies.    We select a sample of AGNs for which $\mbh$ has been independently determined to high accuracy by state-of-the-art techniques: dynamical modeling of the reverberation signal and spatially resolving the broad-line region with VLTI/GRAVITY.    Using IFU observations, we spatially map the host galaxy stellar kinematics across the galaxy and bulge effective radii.    We find that that the dynamically hot component of galaxy disks correlates with $\mbh$ ; however, the correlations are tightest for aperture-integrated $\sigstar$ measured across the bulge.    Accounting for the different $\mbh$ distributions, we demonstrate -- for the first time -- that AGNs follow the same $\mbh$ - $\sigstar$ and $\mbh$ - $\mbulgedyn$ relations as quiescent galaxies.    We confirm that the classical approach of determining the virial factor as sample-average, yielding ${\rm log }f= 0.65 \pm 0.18$ , is consistent with the average $\f$ from individual measurements.    The similarity between the underlying scaling relations of AGNs and quiescent galaxies implies that the current AGN phase is too short to have altered BH masses on a population level. These results strengthen the local calibration of $\f$ for measuring single-epoch $\mbh$ in the distant Universe.

</div>

<div id="div_fig1">

<img src="tmp_2411.02488/./Figures/03_Results/Mbh-sigma_corr.png" alt="Fig16.1" width="50%"/><img src="tmp_2411.02488/./Figures/03_Results/Mbh-Mdyn_corr.png" alt="Fig16.2" width="50%"/>

**Figure 16. -** *\mbh-host-galaxy scaling relations of AGNs and quiescent galaxies.*(Top Left)
    Colored data points show AGNs hosted by LTGs (circles) and ETGs (squares), with the best-fitting observed relation shown as blue stripe (corresponding to row ii in Table \ref{tbl:Results}).
    For AGNs, no clear distinction between the relations of ETGs and LTGs is observed.
    The gray contours show the KH13 sample that is resampled in \mbh to match the AGNs' \mbh-distribution (see Sect. \ref{SubSec:Controlling_selection_effects}). with the fitted relations shown as shaded gray stripe.
    The relation of the \mbh-matched quiescent sample agrees with the AGNs' observed relation, and is significantly flatter than the observed relation of quiescent galaxies shown in the right panel.
    (Top right)
    After empirically matching the \mbh distribution of AGNs (blue contours) to that of quiescent galaxies (KH13 sample, gray data points), both fall onto the same region of the \mbh-\sigstar plane. The best-fitting relations of AGNs are shown as blue and gray stripes, and correspond to the relations in row corresponds to row (v) and (i) respectively, in Table \ref{tbl:Results}.
    (Bottom left)
    The same for the \mbh-\mbulgedyn relation, with the observed AGN listed in row (xii) of Table \ref{tbl:Results}.
    After matching the \mbh distribution, the relations of AGNs and quiescent galaxies are indistinguishable.
    (Bottom right) Same for matching the AGNs to the distribution of quiescent. The \mbh and \mbulgedyn relation correspond to row (xiv) of Table \ref{tbl:Results}.
     (*fig:MBH-scaling-relations*)

</div>
<div id="div_fig2">

<img src="tmp_2411.02488/./Figures/03_Results/disp_reff_bulge.png" alt="Fig1" width="100%"/>

**Figure 1. -** *Radial profile of the spatially resolved stellar velocity dispersion \sigspat across AGN host-galaxy bulges.*
   We measure the spatially resolved stellar dispersion \sigspat in concentric annuli centered on the AGN position.
   Values of \sigspat are normalized to the value measured at the bulge effective radius $R_{\rm eff}^{\rm bulge}$.
   Dashed lines show the spatially resolved \sigspat of individual AGN host galaxies, shaded green regions the 16$^{th}$ to 84$^{th}$ percentile range of the stacked profile.
    (*fig:disp_radial*)

</div>
<div id="div_fig3">

<img src="tmp_2411.02488/./Figures/03_Results/sig_parameters.png" alt="Fig2" width="100%"/>

**Figure 2. -** *Comparing methods for measuring stellar velocity dispersion.*
   Aperture-integrated dispersion measured over $R_{\rm eff}^{\rm gal}$ is shown on the x-axis. On the y-axis, we show the aperture integrated dispersion over $R_{\rm eff}^{\rm bulge}$(blue) and the spatially resolved dispersion over $R_{\rm eff}^{\rm gal}$(orange), respectively.
   Compared to the one-to-one correlation, denoted by the dash-dotted line, the mean bulge dispersion is on average slightly higher than the dispersion measured across the galaxy. In contrast, spatially resolving the kinematics results in significantly lower dispersion.
    (*fig:sig_parameters*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.02488"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\green}[1]{\textcolor{green}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\orange}[1]{\textcolor{orange}{#1}}$
$\newcommand{\magenta}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\MBH}{M_\mathrm{BH}}$
$\newcommand{\MBHm}{M_\mathrm{BH}}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Msunm}{\mathrm{M}_\odot}$
$\newcommand{\MS}{M_\star}$
$\newcommand{\MSm}{M_\star}$
$\newcommand{\MBHMS}{M_\mathrm{BH}--M_\star}$
$\newcommand{\MBHDm}{\rho_\mathrm{BH}}$
$\newcommand{\MBHDsm}{\rho_\mathrm{BH,\star}}$
$\newcommand{\MBHD}{\rho_\mathrm{BH}}$
$\newcommand{\MBHDs}{\rho_\mathrm{BH,\star}}$
$\newcommand{\MAG}{\mathcal{M}}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# The Soltan argument at $z=6$:\ UV-luminous quasars contribute less than 10\% to early black hole mass growth

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.03184-b31b1b.svg)](https://arxiv.org/abs/2411.03184)<mark>Appeared on: 2024-11-06</mark> -  _Submitted to Open Journal of Astrophysics; 10 pages, 5 figures_

</div>
<div id="authors">

<mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We combine stellar mass functions and the recent first JWST-based galaxy--black hole scaling relations at $z=6$ to for the first time compute the supermassive black hole (SMBH) mass volume density at this epoch, and compare this to the integrated SMBH mass growth from the population of UV-luminous quasars at $z>6$ .We show that even under very conservative assumptions almost all growth of supermassive black hole mass at $z>6$ does not take place in these UV-luminous quasars, but must occur in systems obscured through dust and/or with lower radiative efficiency than standard thin accretion disks.The `Sołtan argument' is not fulfilled  by the known population of bright quasars at $z>6$ : the integrated SMBH mass growth inferred from these largely unobscured active galactic nuclei (AGN) in the early Universe is by a factor $\ge$ 10 smaller than the total black hole mass volume density at $z=6$ .This is valid under a large range of assumption about luminosity and mass functions as well as accretion modes, and is likely still at least a factor $>$ 2 smaller when accounting for the known obscuration fractions at this epoch.The resulting consequences are: $>$ 90 \% , possibly substantially more, of SMBH-buildup in the early Universe does not take place in luminous unobscured quasar phases, but has to occur in obscured systems, with dust absorbing most of the emitted UV--visible AGN emission, potentially with accretion modes with super-Eddington specific accretion rates. This is consistent with short lifetime arguments for luminous quasar phases from quasar proximity zone studies and clustering. This would remove the empirical need for slow SMBH growth and hence exotic `high-mass seed' black holes at early cosmic time. It also predicts a large population of luminous but very obscured lower-mass quasars at $z>6$ , possibly the JWST `Little Red Dots'. This finding might severe impact on how we will diagnose SMBH growth at $z=7$ --15 in the future.

</div>

<div id="div_fig1">

<img src="tmp_2411.03184/./figures/Qlumdensity_V6.png" alt="Fig1" width="100%"/>

**Figure 1. -** Bolometric quasar luminosity volume density for largely unobscured UV-luminous quasars at $z=6$, as function of lower cutoff luminosity for integrating the QLF. Black lines show the values for the S23 QLF, green lines for the M18 QLF. Dashed lines use a constant bolometric correction, solid lines a luminosity-dependent BC. (*fig:lumdensity*)

</div>
<div id="div_fig2">

<img src="tmp_2411.03184/./figures/LFevo_V3.png" alt="Fig2" width="100%"/>

**Figure 2. -** Fast evolution of the quasar luminosity function before $z=6$. The curves are normalised to 1 at $z=6$ and show the evolution for three different parameters $k$ as $(1+z)^k$, with values $k=-0.70$(solid line), $k=-0.78$(dashed line), and an extreme $k=-0.5$(dotted) line as a general comparison. The area under the curve is the effective time multiplier to convert the luminosity volume density at $z=6$ into a cumulated emitted energy by quasars from $z=\infty$ to $z=6$. (*fig:lfevo*)

</div>
<div id="div_fig3">

<img src="tmp_2411.03184/./figures/massfuncts_V4.png" alt="Fig3" width="100%"/>

**Figure 3. -** Stellar mass functions (dashed lines) and inferred SMBH mass functions (solid lines). Blue lines refer to the ST21 MF, black lines to the WE24 MF. We note that in both MFs $\MS$ is only constrained by data out to $\log($\MS$m/M_\odot)\sim10.6$. While the massive end does not contribute substantially to the total mass, the notable existence of $\log($\MBH$m)\sim9$ quasars at $z=6$ is not at odds with this. (*fig:massfncts*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.03184"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

242  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
